In [1]:
import pandas as pd 

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [3]:
# aggregate = [
#   {
#     "$match": {
#       "name": {
#         "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
#         "$options": "i"
#       },
#       "data_description.project_name": "Brain Computer Interface",
#       "session.session_type": "BCI single neuron stim"
#     }
#   },
#   {
#     "$project": {
#       "name": 1,
#       "subject_id": "$data_description.subject_id",
#       "session_time": "$session.session_start_time", 
#       "session_type": "$session.session_type", 
#       "genotype": "$subject.genotype", 
#       "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
#       "session_number": {
#         "$filter": {
#           "input": "$session.stimulus_epochs",
#           "as": "epoch",
#           "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
#         }
#       }
#     }
#   },
#   {
#     "$project": {
#       "name": 1,
#       "subject_id": 1,
#       "session_time": 1, 
#       "session_type": 1,
#       "genotype": 1,
#       "virus_injection_materials": 1,
#       "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] }
#     }
#   },
#   {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
#   {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
# ]

# records = docdb_api_client.aggregate_docdb_records(
#     pipeline = aggregate,
# )

In [3]:
aggregate = [
  {
    "$match": {
      "name": {
        "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
        "$options": "i"
      },
      "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim",
      "processing.processing_pipeline.process_start_time":{"$gte":"2025-05-10"}
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_time": "$session.session_start_time", 
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      },
     'ophys_fov': {
            '$map': {
                'input': '$session.data_streams',
                'as': 'stream',
                'in': {
                    'ophys_fov_notes': {
                        '$map': {
                            'input': '$$stream.ophys_fovs',
                            'as': 'fov',
                            'in': '$$fov.notes'
                        }
                    }
                }
            }
        }
        
        
        
      # "ophy_fov": {
      #   "$filter": {
      #     "input": "$session.data_streams",
      #     "as": "stream",
      #     "cond": { "$ne": ["$$stream.ophys_fov", "test"] }
      #   }
      # }
    }
  },
  # {
  #   "$project": {
  #     "name": 1,
  #     "subject_id": 1,
  #     "session_time": 1, 
  #     "session_type": 1,
  #     "genotype": 1,
  #     "virus_injection_materials": 1,
  #     "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] },
  #     "ophys_fov": {"$arrayElemAt": ["$session.data_stream.ophys_fov.notes",0] },
  #     # "ophy_fov": { "$arrayElemAt": ["$ophy_fov.ophy_fov", 0] }
  #   }
  # },
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [13]:
aggregate = [
  {
    "$match": {
      # "name": {
      #   "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
      #   "$options": "i"
      # },
      # "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim",
      "data_description.data_level": "derived",
      "processing.processing_pipeline.data_processes.start_date_time":{"$gte":"2025-05-09"}
    }
  },
  {
    "$project": {
      "name": 1,
    }
  }
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

print(len(records))

27


In [33]:
aggregate = [
  {
    "$match": {
      # "name": {
      #   "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
      #   "$options": "i"
      # },
      # "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim",
      "data_description.data_level": "derived",
      "processing.processing_pipeline.data_processes.start_date_time":{"$gte":"2025-05-09"}
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_time": "$session.session_start_time", 
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      },
      'ophys_fov': {
            '$map': {
                'input': '$session.data_streams',
                'as': 'stream',
                'in': {
                    '$map': {
                            'input': '$$stream.ophys_fovs',
                            'as': 'fov',
                            'in': '$$fov.notes'
                    }
                }
            }
        }

    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_time": 1, 
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] },
      "ophys_fov": 1,
    }
  },
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}}
  
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)


In [35]:
df = pd.DataFrame(records)
df.drop_duplicates(inplace=True)
df

,_id,name,subject_id,session_time,session_type,genotype,virus_injection_materials,ophys_fov,session_number
0,da9c7414-1bf5-4a0d-a796-3e58cc8d5d3e,single-plane-ophys_754303_2025-01-23_19-48-32_...,754303,2025-01-23T18:36:30.521000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,3.0
5,db30abf5-90a9-4e3f-a927-a0a9657ba5b1,single-plane-ophys_754303_2025-01-22_18-22-38_...,754303,2025-01-22T16:59:35.618000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,2.0
10,0fd64eee-aa51-42e0-b37b-32ef53a5263c,single-plane-ophys_766719_2025-01-22_16-45-18_...,766719,2025-01-22T15:36:39.563000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,2.0
15,c0105605-b166-4db7-bcaf-e7a756d62354,single-plane-ophys_754303_2025-01-31_15-13-50_...,754303,2025-01-31T15:13:50.595999-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,8.0
20,0b6a3ab2-c50c-429a-9588-1e6992ded444,single-plane-ophys_731015_2025-01-28_18-56-35_...,731015,2025-01-28T17:40:57.996000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,22.0
26,e19d57eb-048c-49fb-a564-9f23b3205c3d,single-plane-ophys_731015_2025-01-24_20-00-44_...,731015,2025-01-24T18:41:22.550000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,20.0
31,7fe2130f-bfaa-4d5c-8f42-70ea132c0e2a,single-plane-ophys_772414_2025-01-23_16-03-51_...,772414,2025-01-23T14:40:12.962999-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,3.0
36,d593d409-dff3-4381-be2a-9d3290bfe355,single-plane-ophys_766719_2025-01-27_18-25-39_...,766719,2025-01-27T17:02:50.868000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,5.0
41,57178d7b-5c87-4376-933a-769eeabcdd19,single-plane-ophys_731015_2025-01-31_20-37-19_...,731015,2025-01-31T20:37:19.623000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,23.0
47,5c307543-bf11-4a82-bcdf-85f0886d41cb,single-plane-ophys_731015_2025-01-28_17-40-57_...,731015,2025-01-28T17:40:57.996000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,22.0


In [6]:
name = [] 
subject_id = [] 
session_time = [] 
session_type = [] 
genotype = []
virus = []
session_number = []

for record in records: 
    name.append(record["name"])
    subject_id.append(record["subject_id"])
    session_time.append(record["session_time"])
    session_type.append(record["session_type"]) 
    genotype.append(record["genotype"])
    virus.append(record["virus_injection_materials"][0])
    session_number.append(record["session_number"])
    
    
bci_df = pd.DataFrame() 
bci_df["name"] = name 
bci_df["subject_id"] = subject_id
bci_df["session_time"] = session_time
bci_df["session_type"] = session_type
bci_df["genotype"] = genotype
bci_df["virus"] = virus
bci_df["session_number"] = session_number

In [7]:
bci_df

,name,subject_id,session_time,session_type,genotype,virus,session_number
0,single-plane-ophys_772414_2025-01-28_16-19-17,772414,2025-01-28T14:55:51.847000-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,6.0
1,single-plane-ophys_740369_2025-01-09_17-18-37_...,740369,2025-01-09T16:01:04.455000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,22.0
2,single-plane-ophys_772414_2025-01-24_14-49-14_...,772414,2025-01-24T13:30:13.734999-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,4.0
3,single-plane-ophys_772414_2025-01-23_16-03-51,772414,2025-01-23T14:40:12.962999-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,3.0
4,single-plane-ophys_772414_2025-01-28_16-19-17_...,772414,2025-01-28T14:55:51.847000-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,6.0
...,...,...,...,...,...,...,...
102,single-plane-ophys_767715_2025-02-06_18-59-24_...,767715,2025-02-06T18:59:24.815999-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,4.0
103,single-plane-ophys_754303_2025-01-27_20-01-31,754303,2025-01-27T18:46:54.109999-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,5.0
104,single-plane-ophys_766719_2025-01-27_18-25-39_...,766719,2025-01-27T17:02:50.868000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,5.0
105,single-plane-ophys_772414_2025-01-27_15-57-05_...,772414,2025-01-27T14:23:26.693000-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,5.0


In [7]:
bci_df.to_csv('/data/metadata.csv', index= False) 